In [ ]:
# !pip install pyobb

In [25]:
import os
import sys
import yaml
import copy
import warnings
import importlib as imp
from datetime import datetime
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# import pcl
import pyntcloud

sys.path.append('/home/jovyan/work/obstacle-detection/')
# sys.path.append('/home/jovyan/work/boxes_vis/git/obstacle-detection/')

In [2]:
with open('../config.yaml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

In [3]:
obstacle_dct = config['obstacles']
leaning_map_obstacles = config['learning_map']
mapped_obstacles_lst = list(set([leaning_map_obstacles[id] for id in obstacle_dct.keys()]))

In [4]:
import sys
sys.path.append("..")
from model.tasks.semantic.infer2 import get_user 

In [5]:
%cd ..
user = get_user( "squeezeseg")

/home/jovyan/work/obstacle-detection
----------
INTERFACE:
model squeezeseg
----------

Opening arch config file from squeezeseg
Opening data config file from squeezeseg
model folder exists! Using model from squeezeseg
parser started
Using SqueezeNet Backbone
Depth of backbone input =  5
Original OS:  16
New OS:  16
Strides:  [2, 2, 2, 2]
Decoder original OS:  16
Decoder new OS:  16
Decoder strides:  [2, 2, 2, 2]
Total number of parameters:  915540
Total number of parameters requires_grad:  915540
Param encoder  724032
Param decoder  179968
Param head  11540
Successfully loaded model backbone weights
Successfully loaded model decoder weights
Successfully loaded model head weights
Infering in device:  cpu


In [56]:
%%time 
num = '066'
bin_path = 'dataset/sequences/00/velodyne/000' + num + '.bin'
scan = np.fromfile(bin_path, dtype=np.float32)
scan = scan.reshape((-1, 4))[:, :3]

CPU times: user 1.48 ms, sys: 6.58 ms, total: 8.06 ms
Wall time: 20.3 ms


In [57]:
from pipeline import pipeline
from pipeline import common
pipeline = imp.reload(pipeline)

In [58]:

%%time
params = {'roi_x_min': 0, 'roi_x_max': 45,
          'roi_y_min': -14, 'roi_y_max': 14,
          'roi_z_min': -2, 'roi_z_max': 1,
          'eps': 0.65, 'min_samples': 6, 'leaf_size': 120, 
          'proc_labels': False}  
start_time = datetime.now()
label = user.infer(bin_path)
print('Segmentation time: {}'.format((datetime.now() - start_time).total_seconds()))
clusters, cluster_data, time_exec = pipeline.pipeline(scan, label, mapped_obstacles_lst, verbose=True, 
                                    exec_time=True, **params)

Infered scan  dataset/sequences/00/velodyne/000066.bin in 0.7214879989624023 sec
Segmentation time: 0.800246
Execution time:

 - ROI filtering: 0.01097s

 - Filtering obstacles: 0.00239s

 - Voxel grid: 0.00000s

 - Clustering: 0.28638s

 - Min-max cluster points: 0.24257s 

CPU times: user 3.74 s, sys: 280 ms, total: 4.02 s
Wall time: 1.34 s


In [ ]:
import pyntcloud
test_cloud = pyntcloud.PyntCloud(cluster_data)
test_cloud.plot(use_as_color='cluster_id', cmap="cool")

![lol](img/seg_od_clustering.png)

In [50]:
np.savetxt('examples/000066.bbox', clusters)

![lol](img/seg_od_clustering_bbox_000066.png)